### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Format all floats as currency, override in case of %s
pd.options.display.float_format = '${:,.2f}'.format

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Get copy of df with unique SN values
purchase_data_copy = purchase_data.drop_duplicates(subset='SN')

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


## Player Count

* Display the total number of players


In [2]:
Player_Count = pd.DataFrame(data={'Total Players': [len(purchase_data.SN.unique())]})
Player_Count

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# Declare/define dict for Purchasing Analysis
purchasing_analysis = {'Number of Unique Items': len(purchase_data['Item Name'].unique()),
                       'Average Price': purchase_data.Price.mean(),
                       'Total Number of Purchases': purchase_data['Purchase ID'].count(),
                       'Total Revenue': purchase_data.Price.sum()}
# Convert the dict to a dataframe
purchasing_analysis = pd.DataFrame(data=purchasing_analysis, index=[0])
purchasing_analysis

,Number of Unique Items,Average Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# Define Gender summary df
gender_demo = pd.DataFrame(columns=['Total Count', 'Percentage of Players'])
gender_demo['Total Count'] = purchase_data_copy.Gender.value_counts()
gender_demo['Percentage of Players'] = gender_demo['Total Count'] / gender_demo['Total Count'].sum()

# Format percentage column to 2 decimals
gender_demo.style.format({'Percentage of Players': '{:.2%}'})

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Declare df
purchasing_gender_analysis = pd.DataFrame()
# Define df grouped by gender
purchasing_gender_analysis['Purchase Count'] = purchase_data.groupby(['Gender']).count()['Purchase ID']
purchasing_gender_analysis['Total Purchase Value'] = purchase_data.groupby(['Gender']).sum()['Price']
purchasing_gender_analysis.insert(1, 'Average Purchase Price',  purchasing_gender_analysis['Total Purchase Value'] / purchasing_gender_analysis['Purchase Count'])
purchasing_gender_analysis['Avg Total Purchasing per Person'] = purchasing_gender_analysis['Total Purchase Value'] / gender_demo['Total Count']
purchasing_gender_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchasing per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Define bins and the corresponding labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Using copy with duplicates removed, cut data, count by label, and assign to DataFrame
age_demo = pd.cut(purchase_data_copy['Age'], bins,labels=labels).value_counts()
age_demo = pd.DataFrame(age_demo).sort_index()

# Calculate percentage, rename column, display with % format
age_demo['Percentage of Players'] = age_demo['Age'] / age_demo['Age'].sum()
age_demo = age_demo.rename(columns = {'Age': 'Total Count'})
age_demo.style.format({'Percentage of Players': '{:.2%}'})

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# Declare df
age_analysis = pd.DataFrame()

# Cut data using bins/labels from age demo
age_bins = pd.cut(purchase_data['Age'], bins,labels=labels)

age_analysis['Purchase Count'] = age_bins.value_counts().sort_index()
age_analysis['Total Purchase Value'] = purchase_data['Price'].groupby(age_bins).sum()
age_analysis.insert(1,'Average Purchase Price', age_analysis['Total Purchase Value'] / age_analysis['Purchase Count'])
age_analysis['Avg Total Purchase per Person'] = age_analysis['Total Purchase Value'] / age_demo['Total Count']

age_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# Declare df
top_spenders = pd.DataFrame()

# Group data by SN
sn_groups = purchase_data.groupby(['SN'])

top_spenders['Purchase Count'] = sn_groups['Purchase ID'].count()
top_spenders['Total Purchase Value'] = sn_groups['Price'].sum()
top_spenders.insert(1, 'Average Purchase Price', top_spenders['Total Purchase Value'] / top_spenders['Purchase Count'])

top_spenders.sort_values(by=['Total Purchase Value'], ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Declare df
popular_items = pd.DataFrame()

# Group data by SN
item_id_name_groups = purchase_data.groupby(['Item ID', 'Item Name'])

popular_items['Purchase Count'] = item_id_name_groups['Purchase ID'].count()
popular_items['Total Purchase Value'] = item_id_name_groups['Price'].sum()
popular_items.insert(1, 'Item Price', popular_items['Total Purchase Value'] / popular_items['Purchase Count'])

# Display head sorted by Purchase Count, sub-sorted by Index
popular_items.sort_values(by=['Purchase Count', 'Item ID'], ascending=False).head(10)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# Display head sorted by Total Purchase Value, sub-sorted by Index
popular_items.sort_values(by='Total Purchase Value', ascending=False).head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


# Trends
* While men made up the bulk of the player base, women tended to spend a little more on average. 
* Nine out of ten of the most popular items cost more than the average item.
* The most profitable items tended to correlate with the most popular items; profitability in this case driven by number of purchases rather than base price.
* There are no 'whales' in the top spenders, that is, most of the money made on items comes from a diffuse player base, rather than a small number of users.
* The largest player base is in the 20-24 age range, while also seeing strong involvement from the 15-19 and 25-29 range.

# Suggested Actions
* At this time, growing the player base should be the main concern, as the bulk of profit is driven by infrequent purchases from many users.
    * Marketing targeted at women in their late teens and early twenties could help round out the player base. 
* Finding a way to reduce the overall number of items (starting with the least popular), or push the more popular ones, could result in more purchases.
    * With this strategy, an eye should be kept on the overall number of purchases to verify it remains steady or grows.
